In [11]:
import talib
import pandas as pd
import backtrader as bt

In [12]:
df = pd.read_excel(r'gbp_usd_1h.xlsx')

In [13]:
df['kama'] = talib.KAMA(real=df['close'], timeperiod=24)

In [14]:
df['atr'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=21)

In [16]:
class CustomPandasData(bt.feeds.PandasData):
    lines = ('atr', 'kama')  # add new lines
    params = (
        ('atr', -1),
        ('kama', -1),
    )

In [27]:
class MAStrategy1(bt.Strategy):

    def __init__(self):
        self.trades = []
        self.orders = []
        self._open_trades = {}

        self.call_counter = 0

    def calculate_volume(
            self,
            entry_price: float,
            stop_price: float,
            balance: int = 10000,
            risk: int = 1,
    ):
        self.call_counter += 1
        risk_amount = balance * (risk / 100)
        per_unit_risk = abs(entry_price - stop_price)
        units = risk_amount / per_unit_risk
        return int(units)

    def next(self):
        cal_ma_status = lambda close, ma: 'upper' if close > ma else 'under'

        ma = cal_ma_status(self.data.close[0], self.data.kama[0])

        if ma == "upper":
            price = self.data.close[0]
            entry_price = price
            stop_loss = entry_price - self.data.atr[0] * 4.8
            take_profit = entry_price + self.data.atr[0] * 2.4
            size = self.calculate_volume(
                entry_price,
                stop_loss,
                risk=2
            )
            self.buy_bracket(
                price=entry_price,
                stopprice=stop_loss,
                limitprice=take_profit,
                size=size,
                tradeid=1
            )

        # if current_ma_status == "under":
        #     price = self.data.close[0]
        #     entry_price = price
        #     stop_loss = entry_price + self.data.atr[0] * 2.4
        #     take_profit = entry_price - self.data.atr[0] * 4.8
        #     size = self.calculate_volume(
        #             entry_price,
        #             stop_loss,
        #             risk=2
        #         )
        #     self.sell_bracket(
        #         price=entry_price,
        #         stopprice=stop_loss,
        #         limitprice=take_profit,
        #         size=size
        #     )

    def notify_trade(self, trade: bt.trade.Trade):
        self.trades.append(trade)

    def notify_order(self, order):
        self.orders.append(order)




In [28]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [29]:
data = CustomPandasData(dataname=df.set_index("datetime"))

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(MAStrategy1)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')
cerebro.broker.setcash(10000)
cerebro.broker.set_coo(True)
cerebro.broker.setcommission(commission=0.0, leverage=500)
result = cerebro.run()

-5228.768274050385